In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

SEED = 1337
# !!you should use that seed in all models which accepts random_state

In [24]:
df = pd.read_csv('./train.csv', index_col=0)
df.head()
#dfList = df['Fare'].tolist()
#print(dfList)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
# TODO 1:
# encode Sex column
# hint: use LabelEncoder
from sklearn.preprocessing import LabelEncoder

df['Sex'] = LabelEncoder().fit_transform(df.Sex)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [26]:
# print columns with missing values
for name in df.columns:
    print(name, np.sum(pd.isnull(df[name])))

Survived 0
Pclass 0
Name 0
Sex 0
Age 177
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 687
Embarked 2


In [27]:
# Replace missing values with "unkown"
df.loc[df.Cabin.isnull(), 'Cabin'] = 'unknown'

# fill Fare with median value
df.loc[ np.isnan(df['Fare']), 'Fare'] = df['Fare'].median()

# TODO 2
# Replace missing values in Embarked column with most common port
# hint: use Series.mode method

df.loc[ df.Embarked.isnull(), 'Embarked' ] = df['Embarked'].mode()[0]
df.head()
#dfList = df['Embarked'].tolist()
#print(dfList)
#df[df.columns[10]]
#print(df.to_string())

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,unknown,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,unknown,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,unknown,S


In [28]:
# TODO 3
# train Random Forest regression model to fill missing values in Age column
# use  100 estimators, set oob_score in True, other params by default
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

age_columns = ['Age', 'Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 
       'Fare', 'Embarked']

age_df = df[age_columns]

age_df['Embarked'] = preprocessing.LabelEncoder().fit_transform(age_df['Embarked'])
train_df = age_df.loc[age_df.Age.notnull()]
test_df = age_df.loc[age_df.Age.isnull()]
print('train age', train_df.shape[0])
print('test age', test_df.shape[0])


regr = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=SEED)
#y_train = train_df.Age.values
y_train = train_df.values[:,0]
x_train = train_df.values[:, 1::]
#dfList = train_df.Embarked.tolist()
#print(dfList)
#print(train_df.Embarked)
regr.fit(x_train, y_train)
#regr.predict(train_df)
predictedAges = regr.predict(test_df.values[:, 1::])
df.loc[df.Age.isnull(), 'Age'] = predictedAges

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


train age 714
test age 177


In [11]:
#Question 1: what is the oob_score of trained model?
#Score of the training dataset obtained using an out-of-bag estimate.

In [29]:
# TODO 4
# Create a dataframe of dummy variables for each distinct value of Embarked
# hint: use DataFrame.get_dummies method
from sklearn import preprocessing

df_dummy = pd.get_dummies( df['Embarked'] )
#print(df_dummy)

# Rename the columns from 'S', 'C', 'Q' to 'Embarked_S', 'Embarked_C', 'Embarked_Q'
# hint: use pandas.rename method
df_dummy = df_dummy.rename(columns={'C': 'Embarked_C', 'Q':'Embarked_Q', 'S': 'Embarked_S'})
#print(df_dummy)

# Add the new dummy variables back to the original data set
# hint: use pandas.concat method
df = pd.concat([df, df_dummy], axis=1)
#print(df)


df['Embarked'] = preprocessing.LabelEncoder().fit_transform(df['Embarked'])

In [22]:
# TODO 5
import re

# create feature for the alphabetical part of the cabin number
# hint: use re module
for cab in df['Cabin']:
    if cab != 'unknown':
        df['CabinLetter'] = df['Cabin'].map(lambda x : re.compile("([a-zA-Z]+)").search(x).group())

# convert the distinct cabin letters with incremental integer values
# hint: use pandas.factorize method
df['CabinLetter'] = pd.factorize(df['CabinLetter'])[0]

df['Cabin'] = preprocessing.LabelEncoder().fit_transform(df['Cabin'])


PassengerId
1      147
2       81
3      147
4       55
5      147
6      147
7      129
8      147
9      147
10     147
11     145
12      49
13     147
14     147
15     147
16     147
17     147
18     147
19     147
20     147
21     147
22     111
23     147
24      13
25     147
26     147
27     147
28      63
29     147
30     147
      ... 
862    147
863     94
864    147
865    147
866    147
867    147
868      6
869    147
870    147
871    147
872    102
873     34
874    147
875    147
876    147
877    147
878    147
879    147
880     70
881    147
882    147
883    147
884    147
885    147
886    147
887    147
888     30
889    147
890     60
891    147
Name: Cabin, Length: 891, dtype: int64


In [30]:
# TODO 6

# Divide all fares into quartiles
# hint: use pandas.qcut
df['Fare_bin'] = pd.qcut(df['Fare'], 4)

# create dummies from the result and add tham to dataset
df_dummy_fare = pd.get_dummies(df['Fare_bin'])
df = pd.concat([df, df_dummy_fare], axis=1)

df = df.drop('Fare_bin', axis=1)

In [31]:
df = df.drop(['Name', 'Ticket'], axis=1)
df.shape

(891, 16)

In [32]:
# TODO 7
# make dimension reduction with PCA, leave enough components to explain 98% variance of the data, 
# other params by default. Project dataset on these components
# Of course, you should not project your target variable
from sklearn.decomposition import PCA
percentage = 0.98
pca = PCA(n_components=percentage)

#X_pca = ?

In [ ]:
#Question 2: how many components are in PCA?

In [ ]:
# TODO 8
# plot feature importances from Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


X = df.drop('Survived', axis=1)
y = df.Survived.values
features_list = X.columns
X = X.as_matrix()
 
# Train Random Forest Classifier on datasert with 5000 trees, other params by default
# get feature importances from model

feature_importance = ?
 
# make importances relative to max importance
feature_importance = feature_importance / feature_importance.max()
 
# A threshold below which to drop features from the final data set. Specifically, this number represents
# the percentage of the most important feature's importance value
fi_threshold = 0.10
 
# Get the indexes of all features over the importance threshold

important_idx = ?
 
# Create a list of all the feature names above the importance threshold
important_features = features_list[important_idx]
 
# Get the sorted indexes of important features
sorted_idx = np.argsort(feature_importance[important_idx])[::-1]
 

pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[important_idx][sorted_idx[::-1]], align='center')
plt.yticks(pos, important_features[sorted_idx[::-1]])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.draw()
plt.show()
 
# Remove non-important features from the feature set
# X = ?

In [ ]:
#Question 3: What are 4 top most important features?

In [ ]:
# stack features

X = np.hstack([X, X_pca])
X.shape

In [ ]:
# TODO 9 
# plot learning curves for Random Forest Model (for train and test)
# hint: look at sklearn.learning_curve module. Look at the docs
# use Random Forest with 500 trees, 10-fold cross-validation for roc_auc scoring.
#

In [ ]:
# TODO 10
# plot roc auc curve for Random Forest for test dataset
# hint: use roc_curve and auc functions from sklearn.metrics
# use Random Forest with 5000 trees

In [ ]:
#Question 4: What is the roc auc score on test dataset ?